In [32]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import pandas as pd
import torch.nn.functional as F
import torch
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader


#Variable declarations
train_data = '/Users/anishkirloskar/Documents/Machine-learning/Projects/Boston-Housing/train.csv'
train_df = pd.read_csv(train_data)
dfh = train_df.head()
target = train_df.iloc[:,14:15]
train_inputs = train_df.iloc[:,1:14]

#Test/inference dataset
test_data = '/Users/anishkirloskar/Documents/Machine-learning/Projects/Boston-Housing/test.csv'
test_df = pd.read_csv(test_data)
df1h = test_df.head()
test_inputs = test_df.iloc[:,1:14]


#Convert df to tensors
target_tensors = torch.Tensor(target.values)
train_tensors = torch.Tensor(train_inputs.values)
test_tensors = torch.Tensor(test_inputs.values)

#Define dataset
train_ds = TensorDataset(train_tensors,target_tensors)

#Define data loader
batch_size = 5
train_dl = DataLoader(train_ds,batch_size,shuffle=True)

loss_fn = F.mse_loss
pred = torch.Tensor()
#loss = loss_fn(model(torch_inputs),torch_target)

#Create a function to train the model
def fit(num_epochs, model, loss_fn, opt):
    for epoch in range(num_epochs):
        for xi,yi in train_dl:
            pred = model(xi)
            loss = loss_fn(pred,yi)
            loss.backward()
            opt.step()
            opt.zero_grad()
    
    print('Trained-model loss: ',loss_fn(model(train_tensors),target_tensors))
    #preds = model(torch_inputs)
    #print("Predictions:",preds)
    #print("Targets:",torch_target)
    print('')

#Define model
class NeuralN1(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.linear1 = nn.Linear(13,75)
        self.linear2 = nn.Linear(75,50)
        self.act1 = nn.ReLU()
        self.linear3 = nn.Linear(50,25)
        #self.act2 = nn.ReLU()
        self.linear4 = nn.Linear(25,12)
        self.linear5 = nn.Linear(12,5)
        self.linear6 = nn.Linear(5,1)
        
    def forward(self,x):
        x = self.act1(self.linear1(x))
        x = self.act1(self.linear2(x))
        #x = self.act1(x)
        x = self.act1(self.linear3(x))
        #x = self.act2(x)
        x = self.act1(self.linear4(x))
        x = self.act1(self.linear5(x))
        x = self.linear6(x)
        return x

model = NeuralN1()
loss = loss_fn(model(train_tensors),target_tensors)
print('Initial loss:', loss)
print('')
opt1 = torch.optim.Adam(model.parameters(), lr = 1e-3)
opt2 = torch.optim.Adagrad(model.parameters(), lr = 1e-5)

fit(700,model,loss_fn,opt1)
pred = model(train_tensors)
#print(pred)
#fit(100,model,loss_fn,opt2)


Initial loss: tensor(640.5911, grad_fn=<MseLossBackward>)

Trained-model loss:  tensor(3.1336, grad_fn=<MseLossBackward>)



In [34]:
projection = model(test_tensors)
test_ID = test_df['ID']
proj = projection.detach().numpy()
proj = map(float,proj)
#print(test_ID)
#print(proj)
#output = pd.DataFrame({'ID': test_ID, 'medv' : proj})
output = pd.DataFrame(list(zip(test_ID, proj)), columns =['ID', 'medv'])
output.to_csv('my_submission.csv', index=False)